In [1]:
import openai
import json


# Add relevant functions
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    weather_info = {
        "location": location,
        "temperature": "72",
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }
    return json.dumps(weather_info)


def get_bank_names(name):
    """Return given bank names"""

    result = {
        "name": name,
    }
    return json.dumps(result)

    
def find_period(start_year, end_year):
    """Extract a period from a sentence"""

    result = {
        "start_year": start_year,
        "end_year": end_year,
    }
    return json.dumps(result)


def get_conversation_category(risk_category):
    """Return a risk category: PD, LGD, EAD, RWA"""

    result = {
        "name": risk_category,
    }
    return json.dumps(result)
    

def run_conversation(user_input, call_type="auto"):
    # Step 1: send the conversation and available functions to GPT
    messages = [{"role": "user", "content": user_input}]
    functions = [
        {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                },
                "required": ["location"],
            },
        },

        {
            "name": "get_bank_names",
            "description": "Extract bank name from the sentence",
            "parameters": {
                "type": "object",
                "properties": {
                    "name": {
                        "type": "string",
                        "description": "something like Deutsche bank, Goldman Sachs",
                    },
                },
                "required": ["name"],
            },
        },

        {
            "name": "find_period",
            "description": "Extract period (start year and end year) from the sentence",
            "parameters": {
                "type": "object",
                "properties": {
                    "start_year": {
                        "type": "string",
                        "description": "Start year of the period",
                    },

                    "end_year": {
                        "type": "string",
                        "description": "End year of the period",
                    },
                },
                "required": ["start_year", "end_year"],
            },
        },

        {
            "name": "get_conversation_category",
            "description": "Return one of the risk categories: PD, LGD, EAD, RWA",
            "parameters": {
                "type": "object",
                "properties": {
                    "risk_category": {
                        "type": "string",
                        "description": "Only one of the option is permitted: PD, LGD, EAD, RWA",
                    },
                },
                "required": ["risk_category"],
            },
        }
        
    ]
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=messages,
        functions=functions,
        function_call=call_type,  # auto is default, but we'll be explicit
    )
    response_message = response["choices"][0]["message"]
    print(f'Answer returned by functions: {response["choices"][0]["message"]["function_call"]["arguments"]}')

    # Step 2: check if GPT wanted to call a function
    if response_message.get("function_call"):
        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors
        available_functions = {
            "get_current_weather": get_current_weather,
            "get_bank_names": get_bank_names,
            "find_period": find_period,
            "get_conversation_category": get_conversation_category,
        }  # only one function in this example, but you can have multiple
        
        function_name = response_message["function_call"]["name"]
        fuction_to_call = available_functions[function_name]
        function_args = json.loads(response_message["function_call"]["arguments"])
        function_response = fuction_to_call(**function_args)

        # Step 4: send the info on the function call and function response to GPT
        messages.append(response_message)  # extend conversation with assistant's reply
        messages.append(
            {
                "role": "function",
                "name": function_name,
                "content": function_response,
            }
        )  # extend conversation with function response
        second_response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-0613",
            messages=messages,
        )  # get a new response from GPT where it can see the function response
        return second_response

In [2]:
user_input = "the period started in 1998 and ended in 2010."
out = run_conversation(user_input)
print(out['choices'][0]['message']['content'])

Answer returned by functions: {
  "start_year": "1998",
  "end_year": "2010"
}
The period you mentioned, starting in 1998 and ending in 2010, spanned 12 years.


In [3]:
user_input = "i want to extract bank names from the following sentence: Deutsche Bank and Swedbank?"
out = run_conversation(user_input)
print(out['choices'][0]['message']['content'])

Answer returned by functions: {
"name": "Deutsche Bank and Swedbank"
}
The bank names extracted from the sentence "Deutsche Bank and Swedbank" are:
- Deutsche Bank
- Swedbank


In [4]:
user_input = "What's the weather like in Boston?"
out = run_conversation(user_input)
print(out['choices'][0]['message']['content'])

Answer returned by functions: {
  "location": "Boston"
}
The weather in Boston is currently sunny and windy with a temperature of 72 degrees Fahrenheit.


In [5]:
user_input = "I would like to discuss LGD modelling"
out = run_conversation(user_input, {"name": "get_conversation_category"})
print(out['choices'][0]['message']['content'])

Answer returned by functions: {
  "risk_category": "LGD"
}
Sure, I'd be happy to discuss LGD (Loss Given Default) modeling with you. LGD is a key component in credit risk analysis and refers to the amount of loss that a lender or investor is likely to incur in the event of default by a borrower.

LGD modeling is a process of estimating the potential losses that may occur in the event of default. It involves determining the recovery rate, which is the proportion of a loan or investment that can be recovered in case of default. LGD modeling takes into account various factors such as collateral value, seniority of the asset, industry conditions, and economic factors that affect the recovery of funds.

There are several approaches to LGD modeling, including statistical models and expert judgment. Statistical models typically use historical data to estimate the potential losses based on factors such as loan characteristics, borrower's creditworthiness, and macroeconomic conditions. Expert j